In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q ultralytics imutils

In [ ]:
from tqdm.auto import tqdm

import os
import requests
import zipfile
import cv2
import math
import matplotlib.pyplot as plt
import glob
import numpy as np
import random

from PIL import Image

In [ ]:
ROOT_DIR = '/kaggle/input/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav'
train_imgs_dir = 'images/train'
train_labels_dir = 'labels/train'
val_imgs_dir = 'images/val'
val_labels_dir = 'labels/val'
test_imgs_dir = 'images/test'
test_labels_dir = 'labels/test'
classes = ['Person', 'Car', 'Bicycle', 'OtherVechicle', 'DontCare']
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels, classes=classes, colors=colors, pos='above'):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    height, width, _ = image.shape
    lw = max(round(sum(image.shape) / 2 * 0.003), 2)  # Line width.
    tf = max(lw - 1, 1) # Font thickness.
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates
        xmin = int(x1*width)
        ymin = int(y1*height)
        xmax = int(x2*width)
        ymax = int(y2*height)

        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
        
        class_name = classes[int(labels[box_num])]

        color=colors[classes.index(class_name)]
        
        cv2.rectangle(
            image, 
            p1, p2,
            color=color, 
            thickness=lw,
            lineType=cv2.LINE_AA
        ) 

        # For filled rectangle.
        w, h = cv2.getTextSize(
            class_name, 
            0, 
            fontScale=lw / 3, 
            thickness=tf
        )[0]

        outside = p1[1] - h >= 3
        
        if pos == 'above':
            p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
            cv2.rectangle(
                image, 
                p1, p2, 
                color=color, 
                thickness=-1, 
                lineType=cv2.LINE_AA
            )  
            cv2.putText(
                image, 
                class_name, 
                (p1[0], p1[1] - 5 if outside else p1[1] + h + 2),
                cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=lw/3.5, 
                color=(255, 255, 255), 
                thickness=tf, 
                lineType=cv2.LINE_AA
            )
        else:
            new_p2 = p1[0] + w, p2[1] + h + 3 if outside else p2[1] - h - 3
            cv2.rectangle(
                image, 
                (p1[0], p2[1]), new_p2, 
                color=color, 
                thickness=-1, 
                lineType=cv2.LINE_AA
            )  
            cv2.putText(
                image, 
                class_name, 
                (p1[0], p2[1] + h + 2 if outside else p2[1]),
                cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=lw/3, 
                color=(255, 255, 255), 
                thickness=tf, 
                lineType=cv2.LINE_AA
            )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_path, label_path, num_samples, classes=classes, colors=colors, pos='above'):
    all_training_images = glob.glob(image_path+'/*')
    all_training_labels = glob.glob(label_path+'/*')
    all_training_images.sort()
    all_training_labels.sort()
    
    temp = list(zip(all_training_images, all_training_labels))
    random.shuffle(temp)
    all_training_images, all_training_labels = zip(*temp)
    all_training_images, all_training_labels = list(all_training_images), list(all_training_labels)
    
    num_images = len(all_training_images)
    
    if num_samples == -1:
        num_samples = num_images
    
    num_cols = 2
    num_rows = int(math.ceil(num_samples / num_cols))
        
    plt.figure(figsize=(10 * num_cols, 6 * num_rows))
    for i in range(num_samples):
        image_name = all_training_images[i].split(os.path.sep)[-1]
        image = cv2.imread(all_training_images[i])
        with open(all_training_labels[i], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label, x_c, y_c, w, h = label_line.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels, classes, colors, pos)
        plt.subplot(num_rows, num_cols, i+1) # Visualize 2x2 grid of images.
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize a few training images.
plot(
    image_path=os.path.join(ROOT_DIR, train_imgs_dir), 
    label_path=os.path.join(ROOT_DIR, train_labels_dir),
    num_samples=8
)

In [ ]:
import yaml

# 1. Tạo file dataset.yaml
data_dict = {
    "train": "/kaggle/input/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/train",
    "val": "/kaggle/input/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/val",
    "test": "/kaggle/input/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/test",
    "nc": 5,
    "names": ['Person', 'Car', 'Bicycle', 'OtherVehicle', 'DontCare']
}

with open("dataset.yaml", "w") as f:
    yaml.dump(data_dict, f, sort_keys=False)


# 2. Tạo file model config (Sửa phần Backbone)
model_dict = {
    "nc": 5, 
    
    "backbone": [
        # [from, repeats, module, args]
        
        # --- THAY ĐỔI Ở ĐÂY: Đổi "convnext_tiny" thành "convnext_small" ---
        # ConvNeXt-Small output channels cuối cùng vẫn là 768 giống Tiny
        [-1, 1, "TorchVision", [768, "convnext_small", "DEFAULT", True, 2, True]], # ### <--- ĐÃ SỬA
        
        [-1, 1, "nn.Identity", []], 
        [-1, 1, "nn.Identity", []], 

        # ------ P3 Branch (Target: 256 channels)
        # ConvNeXt-Small Stage 2 cũng có 192 channels -> Giữ nguyên
        [0, 1, "Index", [192, 4]], 
        [-1, 1, "Conv", [256, 1, 1]], 
        [-1, 1, "nn.Identity", []], # index 5 (P3)

        # --------- P4 Branch (Target: 512 channels)
        # ConvNeXt-Small Stage 3 cũng có 384 channels -> Giữ nguyên
        [0, 1, "Index", [384, 6]], 
        [-1, 1, "Conv", [512, 1, 1]], 
        [-1, 1, "nn.Identity", []], # index 8 (P4)

        # ------------------------- P5 Branch (Target: 1024 channels)
        # ConvNeXt-Small Stage 4 cũng có 768 channels -> Giữ nguyên
        [0, 1, "Index", [768, 8]], 
        [-1, 1, "Conv", [1024, 1, 1]], 
        [-1, 1, "SPPF", [1024, 5]] # index 11 (P5)
    ],
    "head": [
        # Phần Head giữ nguyên hoàn toàn
        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
        [[-1, 8], 1, "Concat", [1]], 
        [-1, 3, "C2f", [512, True]], # index 14

        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
        [[-1, 5], 1, "Concat", [1]], 
        [-1, 3, "C2f", [256, True]], # index 17 (Output P3)

        [-1, 1, "Conv", [256, 3, 2]],
        [[-1, 14], 1, "Concat", [1]], 
        [-1, 3, "C2f", [512, True]], # index 20 (Output P4)

        [-1, 1, "Conv", [512, 3, 2]],
        [[-1, 11], 1, "Concat", [1]], 
        [-1, 3, "C2f", [1024, True]], # index 23 (Output P5)

        [[17, 20, 23], 1, "Detect", ["nc"]] 
    ]
}

with open("yolov8-custom.yaml", "w") as f:
    yaml.dump(model_dict, f, sort_keys=False)

print("Đã tạo file yolov8-custom.yaml với backbone ConvNeXt-Small thành công!")

In [ ]:
from ultralytics import YOLO

yolo_efficientnet_model = YOLO('/kaggle/working/yolov8-custom.yaml')

In [ ]:
yolo_efficientnet_train = yolo_efficientnet_model.train(data="/kaggle/working/dataset.yaml", epochs=50, batch = 16)

In [ ]:
best_efficientnet_model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

In [ ]:
from imutils import paths

log_dir = "/kaggle/working/runs/detect/train"
for image_path in sorted(paths.list_images(log_dir)):
    image = Image.open(image_path)
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.title(os.path.basename(image_path))
    plt.axis('off')
    plt.show()

In [ ]:
!zip -r /kaggle/working/runs.zip runs/

In [ ]:
!ls -lh /kaggle/working/


# Inference on Test Images


In [ ]:
results = best_efficientnet_model(os.path.join(ROOT_DIR, test_imgs_dir), conf=0.5, agnostic_nms=True, iou=0.5, save=True)

In [ ]:
import random
import matplotlib.pyplot as plt

def show_random_result(results):
    random_result = random.choice(results)   
    img = random_result.plot()               

    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.show()




In [ ]:
show_random_result(results)

In [ ]:
show_random_result(results)

In [ ]:

show_random_result(results)

In [ ]:
indices = list(range(len(results)))
random_indices = random.sample(indices, 10)
num_cols = 2
num_rows = 5

plt.figure(figsize=(12 * num_cols, 6 * num_rows))
    
for i, idx in enumerate(random_indices):
    image = results[i].plot()
    plt.subplot(num_rows, num_cols, i+1)
    plt.imshow(image)
    plt.axis('off')
plt.tight_layout()
plt.show()


# Compare predictions with ground truth

In [ ]:
ground_colors = [(255, 0, 0) for _ in range(len(classes))]

In [ ]:
!ls runs/detect

In [ ]:
success = best_efficientnet_model.export(format="onnx")  # export the model to ONNX format
success